Professor Edson Cilos

Linkedin: https://www.linkedin.com/in/edson-cilos-032a66162/

Os códigos foram extraídos do "user guide" do [sklearn](https://scikit-learn.org/stable/auto_examples/applications/plot_face_recognition.html#sphx-glr-auto-examples-applications-plot-face-recognition-py)

Esse material é explicado com detalhes no meu [curso](https://www.udemy.com/course/edson-cilos-ml/?referralCode=2C9C581FAB301BBAE173).

# Objetivo

Nesta atividade vamos aplicar a "Análise de Componentes Principais" para um problema de reconhecimento facial. O conjunto de dados utilizados neste mini-projeto é uma versão preprocessada do conjunto “Labeled Faces in the Wild” (LFW).

**Atenção,** de acordo com os fornecedores dos dados: "Independente da performance de um algoritmo treinado no LFW, a partir desses dados não deve-se concluir que o algoritmo é útil, qualquer que seja a aplicação comercial" (tradução livre).

# Módulos utilizados

In [ ]:
from time import time
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.datasets import fetch_lfw_people
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.utils.fixes import loguniform

# Rápida explorada nos dados

Vamos primeiramente buscar os dados dentro da biblioteca de datasets do sklearn

In [ ]:
lfw_people = fetch_lfw_people(min_faces_per_person=70, color=True)

Vamos examinar as dimensões das imagens, isso será útil para alguns plots

In [ ]:
print(lfw_people.images.shape)

In [ ]:
n_samples, h, w, _ = lfw_people.images.shape

In [ ]:
plt.imshow(lfw_people.images[14]/255)

Vamos carregar novamente os nossos dados, com nova escala nos dados e também sem considerar cores

In [ ]:
lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)
n_samples, h, w  = lfw_people.images.shape

Vamos trabalhar com dados 2D (cada instância é uma matriz de pixels, isto é uma foto)

In [ ]:
X = lfw_people.data
n_features = X.shape[1]

Agora vamos carregar a variável algo

In [ ]:
# the label to predict is the id of the person
y = lfw_people.target 
target_names = lfw_people.target_names
n_classes = target_names.shape[0]

Mostrando na tela as variáveis relevantes

In [ ]:
print("Total dataset size:")
print("n_samples: %d" % n_samples)
print("n_features: %d" % n_features)
print("n_classes: %d" % n_classes)

In [ ]:
print(lfw_people.images.shape)

O número de features 1850 é exatamente a multiplicação de 50 por 37 ! Aqui utilizamos o chamado "stacking" para poder alimentar os dados dentor dos algoritmos de Machine Learning

In [ ]:
50*37 == 1850

# Separando em treino e teste & preparando dados

Vamos separar 25% dos dados para teste final

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Aplicação do PCA

Agora vamos utilizar o PCA no nosso conjunto de dados. Esse procedimento é tipicamente chamado de "eigenfaces" (eigenvalues + faces).

***Você deve notar que o PCA é uma técnicas não supervisionada***, de forma que não utilizaremos os rótulos dos dados

In [ ]:
n_components = 150

print(
    "Extraindo as principais %d eigenfaces de %d faces" % 
    (n_components, X_train.shape[0])
)

t0 = time()

pca = PCA(
    n_components=n_components, 
    svd_solver="randomized", 
    whiten=True).fit(X_train)
  
print("PCA computado em %0.3fs" % (time() - t0))

Vamos usar o nosso modelo, PCA, e reconstruir as imagens (lembra que elas estavam transformadas pelo processo de stacking.

In [ ]:
eigenfaces = pca.components_.reshape((n_components, h, w))

Compare ambas as saídas:

In [ ]:
print("Original: " + str(lfw_people.images.shape))
print("Eigenfaces: " + str(eigenfaces.shape))


Vamos visualizar algumas das "componentes principais" do conjunto de dados

In [ ]:
plt.imshow(eigenfaces[0]/255)

In [ ]:
plt.imshow(eigenfaces[100]/255)

Finalmente, projetamos os dados na base ortonomal do autoespaço (eigenfaces)

In [ ]:
t0 = time()
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)
print("Computado em %0.3fs" % (time() - t0))

# Treinando modelo

Vamos treinar um SVM com diferentes valores de "C" e $\gamma$

In [ ]:
print("Treinando modelo")

t0 = time()

param_grid = {
    "C": loguniform(1e3, 1e5),
    "gamma": loguniform(1e-4, 1e-1),
}

clf = RandomizedSearchCV(
    SVC(kernel="rbf", class_weight="balanced"), param_grid, n_iter=10
)

clf = clf.fit(X_train_pca, y_train)

print("Melhor combinação em 10 iterações: %0.3fs" % (time() - t0))
print("Melhor estimador encontrado na busca aleatória:")
print(clf.best_estimator_)

Revise as vídeos aulas do curso para relemebrar as boas práticas em seleção e validação de modelos. A metodologia adotada aqui é meramente ilustrativa

# Testando modelo

Agora vamos fazer uma avaliação final do modelo. 

Lembramos ainda que para fins de estimação da performance final do modelo, o conjunto de teste só deve ser usado uma única vez. Sendo assim, a metodologia aplicada é meramente ilustrativa e qualquer decisão acerca de "qual é melhor modelo" deve ser feita em etapas anteriores!

In [ ]:
t0 = time()
y_pred = clf.predict(X_test_pca)
print("Feito em in %0.3fs" % (time() - t0))

Vejamos o relatório de classificação:

In [ ]:
print(classification_report(y_test, y_pred, target_names=target_names))

Olha que linda a nossa matriz de confusão:

In [ ]:
ConfusionMatrixDisplay.from_estimator(
    clf, X_test_pca, y_test, 
    display_labels=target_names, xticks_rotation="vertical"
)
plt.tight_layout()
plt.show()

# Avaliação visual

Vamos fazer uma inspeção visual do que fizemos até agora. Para isso é importante definir algumas funções auxiliares para plotarmos os resultados


In [ ]:
def plot_gallery(X_images, titles, h, w, n_row=3, n_col=4):
    """Função auxiliar para plotar a galeria de fotos"""
    plt.figure(figsize=(1.8 * n_col, 2.4 * n_row))
    plt.subplots_adjust(bottom=0, left=0.01, right=0.99, top=0.90, hspace=0.35)
    for i in range(n_row * n_col):
        plt.subplot(n_row, n_col, i + 1)
        plt.imshow(X_images[i].reshape((h, w)), cmap=plt.cm.gray)
        plt.title(titles[i], size=12)
        plt.xticks(())
        plt.yticks(())

In [ ]:
def title(y_pred, y_test, target_names, i):
  """Título das imagens """
  pred_name = target_names[y_pred[i]].rsplit(" ", 1)[-1]
  true_name = target_names[y_test[i]].rsplit(" ", 1)[-1]
  return "Predição: %s\n Real:      %s" % (pred_name, true_name)



Finalmente, vejamos os resultados:

In [ ]:
prediction_titles = [
    title(y_pred, y_test, target_names, i) for i in range(y_pred.shape[0])
]

plot_gallery(X_test, prediction_titles, h, w, 10, 4)